<a href="https://colab.research.google.com/github/KendallScott/QTW/blob/main/Case%20Study%205/Case_Study_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize, OneHotEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from yellowbrick.classifier import ConfusionMatrix
from sklearn.preprocessing import StandardScaler
from numpy import arange
from sklearn.model_selection import GridSearchCV

#from example SVM notebook
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

Prediction on the Action column, for use with a firewall.

Port is like an address that allows connection like a tv channel.


In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/KendallScott/QTW/main/Case%20Study%205/log2.csv")
df

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18
...,...,...,...,...,...,...,...,...,...,...,...,...
65527,63691,80,13237,80,allow,314,192,122,6,15,4,2
65528,50964,80,13485,80,allow,4680740,67312,4613428,4675,77,985,3690
65529,54871,445,0,0,drop,70,70,0,1,0,1,0
65530,54870,445,0,0,drop,70,70,0,1,0,1,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65532 entries, 0 to 65531
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Source Port           65532 non-null  int64 
 1   Destination Port      65532 non-null  int64 
 2   NAT Source Port       65532 non-null  int64 
 3   NAT Destination Port  65532 non-null  int64 
 4   Action                65532 non-null  object
 5   Bytes                 65532 non-null  int64 
 6   Bytes Sent            65532 non-null  int64 
 7   Bytes Received        65532 non-null  int64 
 8   Packets               65532 non-null  int64 
 9   Elapsed Time (sec)    65532 non-null  int64 
 10  pkts_sent             65532 non-null  int64 
 11  pkts_received         65532 non-null  int64 
dtypes: int64(11), object(1)
memory usage: 6.0+ MB


In [6]:
df.head()

,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Action,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,57222,53,54587,53,allow,177,94,83,2,30,1,1
1,56258,3389,56258,3389,allow,4768,1600,3168,19,17,10,9
2,6881,50321,43265,50321,allow,238,118,120,2,1199,1,1
3,50553,3389,50553,3389,allow,3327,1438,1889,15,17,8,7
4,50002,443,45848,443,allow,25358,6778,18580,31,16,13,18


In [7]:
df['Source Port'] = df['Source Port'].astype(str)
df['Destination Port'] = df['Destination Port'].astype(str)
df['NAT Source Port'] = df['NAT Source Port'].astype(str)
df['NAT Destination Port'] = df['NAT Destination Port'].astype(str)

In [8]:
df.describe()

,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
count,6.553200e+04,6.553200e+04,6.553200e+04,6.553200e+04,65532.000000,65532.000000,65532.000000
mean,9.712395e+04,2.238580e+04,7.473815e+04,1.028660e+02,65.833577,41.399530,61.466505
std,5.618439e+06,3.828139e+06,2.463208e+06,5.133002e+03,302.461762,3218.871288,2223.332271
min,6.000000e+01,6.000000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
25%,6.600000e+01,6.600000e+01,0.000000e+00,1.000000e+00,0.000000,1.000000,0.000000
50%,1.680000e+02,9.000000e+01,7.900000e+01,2.000000e+00,15.000000,1.000000,1.000000
75%,7.522500e+02,2.100000e+02,4.490000e+02,6.000000e+00,30.000000,3.000000,2.000000
max,1.269359e+09,9.484772e+08,3.208818e+08,1.036116e+06,10824.000000,747520.000000,327208.000000


In [159]:
value_counts = df['Source Port'].value_counts()

#creating list of columns with fewer than 32 values to remove before turning into categorical column (equals .05% of the rows in this dataset)
droplist = pd.DataFrame(value_counts)
droplist = droplist[droplist['Source Port']< 32]
droplist = list(droplist.index.values)

value_counts_DP = df['Destination Port'].value_counts()

#creating list of columns with fewer than 32 values to remove before turning into categorical column (equals .05% of the rows in this dataset)
droplist_DP = pd.DataFrame(value_counts_DP)
droplist_DP = droplist_DP[droplist_DP['Destination Port']< 32]
droplist_DP = list(droplist_DP.index.values)
droplist_DP

['35894',
 '6889',
 '30317',
 '26296',
 '9571',
 '35253',
 '18375',
 '3478',
 '48637',
 '44512',
 '28189',
 '24748',
 '41615',
 '54532',
 '32468',
 '55783',
 '25',
 '60043',
 '5529',
 '35974',
 '26672',
 '1024',
 '33330',
 '57587',
 '67',
 '8100',
 '35776',
 '17500',
 '23388',
 '17',
 '50432',
 '55108',
 '55777',
 '65008',
 '21',
 '27019',
 '45682',
 '10387',
 '8696',
 '2225',
 '2710',
 '56125',
 '61689',
 '9694',
 '3111',
 '3855',
 '5555',
 '10214',
 '50321',
 '6882',
 '40108',
 '47993',
 '8168',
 '19473',
 '46776',
 '8000',
 '49152',
 '7275',
 '28793',
 '11119',
 '6969',
 '49756',
 '8055',
 '81',
 '19910',
 '49903',
 '51180',
 '37450',
 '52752',
 '20010',
 '7454',
 '46783',
 '28678',
 '41864',
 '8060',
 '60038',
 '66',
 '37',
 '50011',
 '11446',
 '1900',
 '17546',
 '27025',
 '6004',
 '30041',
 '51185',
 '62609',
 '3398',
 '38303',
 '10197',
 '38608',
 '33303',
 '34021',
 '64347',
 '58414',
 '2222',
 '3383',
 '48983',
 '8604',
 '1688',
 '54369',
 '33890',
 '3544',
 '45711',
 '53567',


There are 22,724 distinct Source Ports in this dataset.

In [149]:
i=0

#Replacing low frequency Source Ports (under .05%), with "Dropped" value, to limit the size of the dataframe

while i < len(droplist):
  df['Source Port']=df['Source Port'].replace((droplist[i]),"Dropped")
  print(droplist[i])
  i+=1


19875
13285
44847
30188
6882
1025
57987
8889
6889
50052
40204
25565
48676
51414
8999
55877
10151
54280
50116
47910
21133
19473
43584
50470
27988
52459
17750
51880
45682
51413
993
6712
55056
49767
55600
49001
61550
49708
5222
55199
5938
56624
1027
51244
55786
56127
55836
49812
10669
49516
55179
42252
49364
43129
56658
49501
56388
56703
56661
49514
49369
49531
65515
55893
54998
55043
49845
49701
55605
55072
65115
49631
65487
49518
64811
57275
64605
49558
55704
65176
56704
50280
55620
50095
58022
56629
56586
58338
49913
64677
55830
56655
49699
56724
65049
56783
56611
49837
64016
55027
55688
11295
55286
49234
49891
57339
50972
64614
56752
55675
64567
57396
65516
55123
49907
49650
65228
56375
55678
64586
9687
49901
55727
56330
57284
55735
56415
56612
49777
49242
49429
55454
55417
56561
49359
60786
56622
56765
64980
55155
49663
55820
56941
64357
56131
65229
56004
49713
56654
49762
57431
49835
49622
50438
57432
56569
56712
56326
56770
64595
64986
56837
49742
65507
50478
49323
65220
43260
4962

In [160]:
i=0

#Replacing low frequency Source Ports (under .05%), with "Dropped" value, to limit the size of the dataframe

while i < len(droplist_DP):
  df['Destination Port']=df['Destination Port'].replace((droplist_DP[i]),"Dropped")
  print(droplist_DP[i])
  i+=1

35894
6889
30317
26296
9571
35253
18375
3478
48637
44512
28189
24748
41615
54532
32468
55783
25
60043
5529
35974
26672
1024
33330
57587
67
8100
35776
17500
23388
17
50432
55108
55777
65008
21
27019
45682
10387
8696
2225
2710
56125
61689
9694
3111
3855
5555
10214
50321
6882
40108
47993
8168
19473
46776
8000
49152
7275
28793
11119
6969
49756
8055
81
19910
49903
51180
37450
52752
20010
7454
46783
28678
41864
8060
60038
66
37
50011
11446
1900
17546
27025
6004
30041
51185
62609
3398
38303
10197
38608
33303
34021
64347
58414
2222
3383
48983
8604
1688
54369
33890
3544
45711
53567
995
50934
14446
50238
63710
53395
61686
33720
10010
49813
60496
47807
25773
55559
16386
16385
31652
39210
16933
28080
37649
65151
62732
65473
40005
9918
55839
53945
4607
14591
16384
25320
20541
35447
9822
62451
40031
56448
30228
7010
44091
55160
57354
30975
27020
4205
8497
45788
36653
3434
14037
1025
40002
24874
49001
7129
20087
27520
54507
31061
17637
46314
47783
56079
38211
52848
12446
44683
30627
3385
14914
16010


In [161]:
# Make dataframe for model input
X = df.loc[:, df.columns != 'Action']

#one hot encoding categorical variables for model
cols = X.columns
num_cols = X._get_numeric_data().columns
cat_cols = list((set(cols) - set(num_cols)))

#creating dataframe of categorical columns
cat_df = X[cat_cols]
cat_df = pd.get_dummies(cat_df, columns=cat_df.columns)

#creating dataframe of numeric columns
scaler = StandardScaler()
num_df = scaler.fit_transform(X[num_cols])

X=cat_df.join(pd.DataFrame(num_df,columns=X[num_cols].columns, index=X[num_cols].index))
X

,Destination Port_0,Destination Port_1,Destination Port_11392,Destination Port_123,Destination Port_1433,Destination Port_15000,Destination Port_15503,Destination Port_161,Destination Port_17372,Destination Port_21247,...,NAT Destination Port_995,NAT Destination Port_9992,NAT Destination Port_9999,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,pkts_received
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.017255,-0.005823,-0.030308,-0.019651,-0.118474,-0.012551,-0.027197
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.016438,-0.005430,-0.029056,-0.016339,-0.161455,-0.009755,-0.023598
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.017244,-0.005817,-0.030293,-0.019651,3.746507,-0.012551,-0.027197
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.016695,-0.005472,-0.029575,-0.017118,-0.161455,-0.010376,-0.024498
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.012773,-0.004077,-0.022799,-0.014001,-0.164761,-0.008823,-0.019550
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.017231,-0.005798,-0.030293,-0.018871,-0.168067,-0.011619,-0.026747
65528,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0.815823,0.011736,1.842607,0.890740,0.036919,0.293149,1.632037
65529,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.017274,-0.005829,-0.030342,-0.019845,-0.217661,-0.012551,-0.027646
65530,0,0,0,0,0,0,0,0,0,0,...,0,0,0,-0.017274,-0.005829,-0.030342,-0.019845,-0.217661,-0.012551,-0.027646


Updates for Source & Destination Port, changed column number from 57,689 down to 31,805 by replacing values which represented less than 99.95% of the data overall (to help make the model more efficient, since that is a requirement for this model to be leveraged).

In [ ]:
X.describe()

In [ ]:
#creating dataframe to find where there is high readmitted rates for a variable
y_alt = df[['Action']]
y_alt = pd.get_dummies(y_alt, columns=y_alt.columns)

In [ ]:
model = LinearSVC(max_iter=100000,tol=0.01)
X_train,X_test, y_train,y_test = train_test_split(X,y_alt)

In [25]:
y_alt

,Action_allow,Action_deny,Action_drop,Action_reset-both
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0
...,...,...,...,...
65527,1,0,0,0
65528,1,0,0,0
65529,0,0,1,0
65530,0,0,1,0
